In [137]:
import random
import numpy as np

- c - вместимость
- p - ценность объектов
- s - оптимальный выбор объектов 
- w - веса объектов

In [138]:
# Читаем данные
def read_file_to_list(filename):
    with open(filename, 'r') as file:
        return [float(line.strip()) for line in file if line.strip()]
    
n = 1

capacity = read_file_to_list(f'benchmarks/P0{n}/p0{n}_c.txt')[0]
profits = read_file_to_list(f'benchmarks/P0{n}/p0{n}_p.txt')
opt_selection = read_file_to_list(f'benchmarks/P0{n}/p0{n}_s.txt')
weights = read_file_to_list(f'benchmarks/P0{n}/p0{n}_w.txt')

In [139]:
print("Вместимость рюкзака (c):", capacity)
print("Ценности предметов (p):", profits)
print("Оптимальный выбор (s):", opt_selection)
print("Веса предметов (w):", weights)

Вместимость рюкзака (c): 165.0
Ценности предметов (p): [92.0, 57.0, 49.0, 68.0, 60.0, 43.0, 67.0, 84.0, 87.0, 72.0]
Оптимальный выбор (s): [1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
Веса предметов (w): [23.0, 31.0, 29.0, 44.0, 53.0, 38.0, 63.0, 85.0, 89.0, 82.0]


In [140]:
# Подсчёт веса и ценности рюкзака
def calculate_backpack(solution, weights, profits):
    total_weight = 0
    total_profit = 0
    
    for i in range(len(solution)):
        if solution[i] == 1:
            total_weight += weights[i]
            total_profit += profits[i]
    
    return total_weight, total_profit

In [141]:
# Уменьшаем вес
def delete_weight(solution):
    w, p = calculate_backpack(solution, weights, profits)
    
    while w > capacity:
        ones_indices = [i for i, x in enumerate(solution) if x == 1]
    
        random_index = random.choice(ones_indices)
        solution[random_index] = 0
        
        w, p = calculate_backpack(solution, weights, profits)
    return solution

In [142]:
# Рандомно заполняем массив нулями и единицами
def random_list():
    
    solution = [random.randint(0, 1) for _ in range(len(weights))]
    solution = delete_weight(solution)

    return solution

solutions = [[],[],[],[],[]]
solutions_len = len(solutions)

for i in range(5):
    solutions[i] = random_list()

In [143]:
print(solutions[0])
print(solutions[1])
print(solutions[2])
print(solutions[3])
print(solutions[4])

w, p = calculate_backpack(solutions[0], weights, profits)
print(p)
w, p = calculate_backpack(solutions[1], weights, profits)
print(p)
w, p = calculate_backpack(solutions[2], weights, profits)
print(p)
w, p = calculate_backpack(solutions[3], weights, profits)
print(p)
w, p = calculate_backpack(solutions[4], weights, profits)
print(p)

[0, 0, 0, 0, 0, 0, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 1, 1, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 1, 0]
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 1, 0, 0]
151.0
110.0
147.0
201.0
176.0


In [133]:
best_solution = []
max_profit = 0

sum_p = 0
probabilities = []

for _ in range(solutions_len):
    w, p = calculate_backpack(solutions[_], weights, profits)
    if p > max_profit:
        max_profit = p
        best_solution = solutions[_]
    sum_p += p
    probabilities.append(p)

for _ in range(solutions_len):
    probabilities[_] = probabilities[_]/sum_p

In [134]:
print(best_solution)
print(max_profit)

[1, 0, 0, 1, 0, 0, 0, 0, 1, 0]
247.0


In [135]:
print(sum_p)
print(probabilities)

782.0
[0.20332480818414322, 0.16240409207161124, 0.170076726342711, 0.1483375959079284, 0.31585677749360613]


In [124]:
# Кроссовер
def crossover(a, b):
    n = len(a)
    part_size = round(n / 3)
    sizes = [part_size, part_size, n - 2 * part_size]
    
    a_parts = []
    b_parts = []
    start = 0
    for size in sizes:
        a_parts.append(a[start:start+size])
        b_parts.append(b[start:start+size])
        start += size
    
    a = a_parts[0] + b_parts[1] + a_parts[2]
    b = b_parts[0] + a_parts[1] + b_parts[2]
    
    return a, b

In [136]:
# Выбираем 2 подсписка на основе вероятностей
selected_indices = np.random.choice(
    len(solutions), 
    size=2, 
    replace=False, 
    p=probabilities
)

selected = [solutions[i] for i in selected_indices]

# Кроссоверим выбранные списки
res_1, res_2 = crossover(selected[0], selected[1])

# Добавляем мутацию flip bit к полученным решениям
def invert_three_elements(result):
    new_result = result.copy()
    indices = random.sample(range(len(result)), 2)

    for index in indices:
        if 0 <= index < len(new_result):
            new_result[index] = 1 - new_result[index]
    return new_result

child_1 = invert_three_elements(res_1)
child_2 = invert_three_elements(res_2)

delete_weight(child_1)
delete_weight(child_2)

# Удаляем выбранные подсписки из исходного списка
for i in sorted(selected_indices, reverse=True):
    del solutions[i]

solutions.append(child_1)
solutions.append(child_2)
    
print("Оставшиеся подсписки:", solutions)

Оставшиеся подсписки: [[1, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0, 1, 0, 0, 0], [0, 0, 1, 0, 0, 0, 1, 0, 0, 0], [1, 1, 0, 0, 0, 0, 0, 0, 1, 0], [1, 0, 1, 0, 0, 0, 0, 1, 0, 0]]


In [146]:
[[0,[]]]*7

[[0, []], [0, []], [0, []], [0, []], [0, []], [0, []], [0, []]]